In [1]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [2]:
# creating dummy data
# The sample data was from a population of 2100 individuals.
# 5% of the population aged <65 years was found to have Covid-19 while 95% didnot.
# 95% of the population aged 65>= years was found to have Covid-19 while 5% didnot.

train_samples = []
train_labels = []

for _  in range(50):
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(1)

    random_older = randint(65, 100)
    train_samples.append(random_older)
    train_labels.append(0)

for _ in range(1000):
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(0)

    random_older = randint(65, 100)
    train_samples.append(random_older)
    train_labels.append(1)

In [3]:
test_samples = []
test_labels = []

for _  in range(50):
    random_younger = randint(13,64)
    test_samples.append(random_younger)
    test_labels.append(1)

    random_older = randint(65, 100)
    test_samples.append(random_older)
    test_labels.append(0)

for _ in range(200):
    random_younger = randint(13,64)
    test_samples.append(random_younger)
    test_labels.append(0)

    random_older = randint(65, 100)
    test_samples.append(random_older)
    test_labels.append(1)

In [4]:
train_samples = np.array(train_samples)
train_labels = np.array(train_labels)

test_samples = np.array(test_samples)
test_labels = np.array(test_labels)

In [7]:
train_samples, train_labels = shuffle(train_samples, train_labels)
test_samples, test_labels = shuffle(test_samples, test_labels)

In [9]:
scaler = MinMaxScaler(feature_range=(0,1))

scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))
scaled_test_samples = scaler.fit_transform(test_samples.reshape(-1,1))

In [11]:
scaled_train_samples

array([[0.81609195],
       [0.25287356],
       [0.1954023 ],
       ...,
       [0.64367816],
       [0.29885057],
       [0.26436782]])

In [13]:
# hyperparameters
devices  = tf.config.experimental.list_physical_devices('CPU')
# tf.config.experimental.set_memory_growth(devices[0], enable=True) # only for gpu devices
lr = 0.0001
optimizer = Adam(learning_rate=lr)
loss = 'sparse_categorical_crossentropy'
metrics = ['accuracy']
epochs = 30
batch_size = 10

In [15]:
# model
model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                32        
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer, loss, metrics)

In [19]:
model.fit(
    x=scaled_train_samples,
    y=train_labels,
    validation_split=0.1,
    # validation_data=valid_data, #> valid data is a tuple containing (x_samples, x_labels) of numpy arrays or tensor data. It can be used instead of validation_split
    batch_size=batch_size,
    epochs=epochs,
    shuffle=True, #! This shuffle occurs after validation_split hence your training data should be shuffled prior to fitting or in an instance where we don't have to split our data.
    verbose=2
)

Epoch 1/30
189/189 - 5s - loss: 0.7404 - accuracy: 0.4302 - val_loss: 0.7146 - val_accuracy: 0.4048 - 5s/epoch - 28ms/step
Epoch 2/30
189/189 - 1s - loss: 0.7042 - accuracy: 0.2921 - val_loss: 0.6913 - val_accuracy: 0.4333 - 766ms/epoch - 4ms/step
Epoch 3/30
189/189 - 1s - loss: 0.6761 - accuracy: 0.5074 - val_loss: 0.6765 - val_accuracy: 0.4333 - 861ms/epoch - 5ms/step
Epoch 4/30
189/189 - 1s - loss: 0.6564 - accuracy: 0.5466 - val_loss: 0.6635 - val_accuracy: 0.5095 - 829ms/epoch - 4ms/step
Epoch 5/30
189/189 - 1s - loss: 0.6386 - accuracy: 0.6011 - val_loss: 0.6500 - val_accuracy: 0.5524 - 817ms/epoch - 4ms/step
Epoch 6/30
189/189 - 1s - loss: 0.6210 - accuracy: 0.6386 - val_loss: 0.6359 - val_accuracy: 0.5857 - 805ms/epoch - 4ms/step
Epoch 7/30
189/189 - 1s - loss: 0.6037 - accuracy: 0.6730 - val_loss: 0.6213 - val_accuracy: 0.6095 - 806ms/epoch - 4ms/step
Epoch 8/30
189/189 - 0s - loss: 0.5857 - accuracy: 0.6958 - val_loss: 0.6043 - val_accuracy: 0.6333 - 326ms/epoch - 2ms/step
Ep

In [20]:
predictions = model.predict(
    x=scaled_test_samples,
    batch_size=batch_size,
    verbose=0 #! show no output
)

In [21]:
predictions = np.argmax(predictions, axis=-1)

In [23]:
# saving the model
model.save('model_1.h5')

In [24]:
# loading the model
from tensorflow.keras.models import load_model

model = load_model('model_1.h5')

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                32        
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.get_weights()

[array([[-0.33747038,  0.42811686, -0.03471029, -0.4783892 , -0.43343067,
         -0.47024465,  0.31906188, -0.08378875, -0.57466906, -0.31035548,
         -0.47595114,  0.79456574,  0.29409453,  0.75208783, -0.23051614,
          0.5916312 ]], dtype=float32),
 array([ 0.        , -0.14411163,  0.        ,  0.        ,  0.        ,
         0.        , -0.11892646,  0.        ,  0.        ,  0.        ,
         0.        ,  0.04843576, -0.1096886 , -0.19272542,  0.        ,
        -0.14828801], dtype=float32),
 array([[ 0.29528973,  0.28838018,  0.31068245,  0.31499448, -0.3124222 ,
         -0.15082705, -0.08352992, -0.14928979,  0.29206756,  0.05470425,
         -0.00999597,  0.06567061, -0.11592229, -0.15490088,  0.18434468,
         -0.1786615 ,  0.17872617,  0.06176409, -0.09139013, -0.17386451,
         -0.08237678, -0.19326167, -0.13679096, -0.02620885,  0.07729125,
          0.24635336, -0.04998836, -0.24200206,  0.32416555,  0.31543997,
         -0.2196016 , -0.28238684],
 

In [27]:
model.optimizer

In [31]:
# save as json
json_string = model.to_json() # only when saving the architecture of the model and not its weights or its training configuration

In [34]:
# model reconstruction from JSON
from tensorflow.keras.models import model_from_json

model_architecture = model_from_json(json_string)

In [36]:
model_architecture.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                32        
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [38]:
# if you only need to save the weights of a model
model.save_weights('model_weights.h5')

In [43]:
model2 = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

In [44]:
model2.get_weights()

[array([[ 0.5713167 ,  0.21958905, -0.24342608,  0.37718558,  0.27057552,
          0.30539918, -0.4294687 , -0.10991684,  0.09596086, -0.13975269,
          0.31878233, -0.52556896, -0.24167296,  0.29639953,  0.21867931,
         -0.09738961]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([[-0.14700381, -0.2999838 , -0.0457648 ,  0.06100646, -0.00792983,
         -0.28740323,  0.2594405 ,  0.23153159,  0.0044153 , -0.31445223,
          0.34198812, -0.2604875 ,  0.1986306 , -0.09160128,  0.29460016,
         -0.3292864 ,  0.06031138,  0.12243262, -0.01476792,  0.09680828,
          0.1086368 ,  0.00933588, -0.03468508,  0.15705398,  0.25672707,
          0.07577792,  0.1698024 ,  0.1543211 , -0.07700431,  0.16739741,
         -0.00209031,  0.09813035],
        [-0.29561758,  0.25542262,  0.23843315,  0.24832669, -0.27227664,
          0.07269016, -0.15837066,  0.15609226, -0.2973085 ,  0.13765168,
          0.22

In [45]:
model2.load_weights('model_weights.h5')

In [46]:
model2.get_weights()

[array([[-0.33747038,  0.42811686, -0.03471029, -0.4783892 , -0.43343067,
         -0.47024465,  0.31906188, -0.08378875, -0.57466906, -0.31035548,
         -0.47595114,  0.79456574,  0.29409453,  0.75208783, -0.23051614,
          0.5916312 ]], dtype=float32),
 array([ 0.        , -0.14411163,  0.        ,  0.        ,  0.        ,
         0.        , -0.11892646,  0.        ,  0.        ,  0.        ,
         0.        ,  0.04843576, -0.1096886 , -0.19272542,  0.        ,
        -0.14828801], dtype=float32),
 array([[ 0.29528973,  0.28838018,  0.31068245,  0.31499448, -0.3124222 ,
         -0.15082705, -0.08352992, -0.14928979,  0.29206756,  0.05470425,
         -0.00999597,  0.06567061, -0.11592229, -0.15490088,  0.18434468,
         -0.1786615 ,  0.17872617,  0.06176409, -0.09139013, -0.17386451,
         -0.08237678, -0.19326167, -0.13679096, -0.02620885,  0.07729125,
          0.24635336, -0.04998836, -0.24200206,  0.32416555,  0.31543997,
         -0.2196016 , -0.28238684],
 